# 실행모드(ESC)
* 커서가 사라집니다.
* a키를 누르면 위에 셀이 생성됩니다.
* b키를 누르면 아래에 셀이 생성됩니다.
* dd키를 누르면 셀이 삭제 됩니다.
* m키를 누르면 마크다운 셀로 변경됩니다.


# 편집모드(Enter)
* 커서가 생깁니다.

In [ ]:
%pwd

'/home/sjt/projects/web/likelion/AIS8'

In [39]:
import os

os.getcwd()

'/home/sjt/projects/web/likelion/AIS8'

In [40]:
# time : 간격을 두고 가져오기 위해 사용
# pandas : 파이썬에서 사용할 수 있는 엑셀과 유사한 데이터분석 도구
# numpy : 파이썬에서 사용할 수 있는 공학용 계산기
# requests : 매우 작은 브라우저로 웹사이트의 내용과 정보를 불러옴
# BeautifulSoup : requests로 가져온 웹사이트의 html 태그를 찾기위해 사용
import time
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs

In [41]:
def get_one_page(page_no):
    base_url = f'https://opengov.seoul.go.kr/civilappeal/list?items_per_page=50&page={page_no}'
    try:
        response = requests.get(base_url)
    except requests.exceptions.HTTPError as errh:
        time.sleep(3)
        try:
            response = requests.get(base_url)
        except requests.exceptions.HTTPError as errh:
            print ("Http Error : ",errh)
    df = pd.read_html(response.text)[0]
    html = bs(response.text)
    text = [i['href'].split('/')[-1] for i in html.select('td.data-title.aLeft > a')]
    try:
        df['내용번호'] = text
    except ValueError as errv:
        print(f'df 길이 : {len(df)} / 리스트 길이 : {len(text)}')
        print('ValueError : ',errv)
    return df

In [42]:
page_no = 1
table_list = []

while True:
    df_one = get_one_page(page_no)
    table_list.append(df_one)
    print(page_no, end=",")
    if len(df_one) < 2 :
        break
    page_no += 1
    time.sleep(0.01)

df = pd.concat(table_list)
df

1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,

In [ ]:
df = df.sort_values(by=['번호'], axis=0).reset_index(drop=True)
df.to_csv('./data/seoul-120/seoul-120-list.csv',index=False)

In [2]:
from urllib.error import HTTPError
import time
import requests
from tqdm import tqdm
tqdm.pandas()
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs

def scraping_error_handle(func):
    try:
        return func
    except HTTPError as errh:
        try:
            time.sleep(2)
            return func
        except:
            print(f'http error : {errh}')
    except ValueError as errv:
        print(f'ValueError : {errv}')

def get_desc(response):
    df_desc = pd.read_html(response.text)[1]
    tb01 = df_desc[[0,1]].set_index(0).T.reset_index(drop=True)
    tb02 = df_desc[[2,3]].set_index(2).T.reset_index(drop=True)
    df_tb = pd.concat([tb01,tb02],axis=1,ignore_index=False)
    return df_tb

@scraping_error_handle
def get_view_page(view_no):
    base_url = f'https://opengov.seoul.go.kr/civilappeal/{view_no}'
    response = requests.get(base_url)
    desc = get_desc(response)
    html = bs(response.text)
    text = html.find('div', {'class' : 'line-all'}).text.strip()
    desc['내용'] = text
    desc['내용번호'] = view_no
    time.sleep(0.01)
    return desc

In [24]:
df = pd.read_csv('./data/seoul-120/seoul-120-list.csv')
df = df[-10:].reset_index(drop=True)
df

,번호,제목,생산일,조회수,내용번호
0,2234,서울시 [우리동네꼼꼼육아정보] 책자 발간,2019-10-23,2275,18955676
1,2235,"[성동구] 반려동물 ""무료상담실"" 운영 안내",2020-08-31,1654,21095911
2,2236,위드유 서울 직장 성희롱.성폭력 예방센터,2020-09-16,1924,21212235
3,2237,마포 뇌병변장애인 비전센터,2021-03-12,1625,22477798
4,2238,[서울산업진흥원] 서울메이드란?,2021-06-29,8168,23194045
5,2239,다자녀가정 실내 바닥매트 지원,2021-08-17,1319,25670204
6,2240,하반기 농부의 시장 일정은 어떻게 되나요?,2022-08-29,647,26695536
7,2241,핑크퐁 서울공원 대탐험 행사에 참여하려면 예약이 필요한가요?,2022-10-05,48,27097033
8,2242,핑크퐁 서울공원 대탐험 행사는 비가 와도 운영하나요?,2022-10-05,129,27097034
9,2243,수도요금 전자계산서 발급 요청시,2022-11-28,121,27328170


In [25]:
view_no = df['내용번호'][0]
get_view_page(view_no)

,원본시스템,작성자(책임자),관리번호,제공부서,생산일,분류,내용,내용번호
0,다산콜센터,120다산콜재단,D0000038442669,서울특별시 여성가족정책실 아이돌봄담당관,2019-10-23,복지,서울시와 서울시육아종합지원센터는 140여 명의 우리동네 보육반장이 직접 발로 뛰어 ...,18955676


In [31]:
view_detail = df['내용번호'].progress_apply(get_view_page)
view_detail = pd.concat(view_detail.to_list()).sort_values('내용번호').reset_index(drop=True)
view_detail

100%|██████████| 10/10 [00:04<00:00,  2.24it/s]


,원본시스템,작성자(책임자),관리번호,제공부서,생산일,분류,내용,내용번호
0,다산콜센터,120다산콜재단,D0000038442669,서울특별시 여성가족정책실 아이돌봄담당관,2019-10-23,복지,서울시와 서울시육아종합지원센터는 140여 명의 우리동네 보육반장이 직접 발로 뛰어 ...,18955676
1,다산콜센터,120다산콜재단,D0000040706778,서울특별시 자치구,2020-08-31,경제,"▣ 반려동물 무료상담실_""무엇이든 물어보시개?"" 운영 안내 구분 내용 ...",21095911
2,다산콜센터,120다산콜재단,D0000040829456,서울특별시 여성가족정책실 여성권익담당관,2020-09-16,복지,업무개요 직장 내 성희롱 지원체계 사각지대에 있는 서울 시민이 안전하고 성평등하...,21212235
3,다산콜센터,120다산콜재단,D0000042114742,서울특별시 복지정책실 복지기획관 장애인복지정책과,2021-03-12,복지,▣ 마포뇌병변장애인 비전센터 운영 구분 내용 목적 학령기 이후 갈...,22477798
4,다산콜센터,120다산콜재단,D0000042894548,서울산업진흥원,2021-06-29,경제,서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해 만들어진 라이프스타일 플랫...,23194045
5,다산콜센터,120다산콜재단,D0000043274644,서울특별시 서초구 주민생활국 여성보육과,2021-08-17,복지,업무개요 다자녀가정 아이들의 안전한 실내 활동과 층간소음 예방을 위해 실내 바닥...,25670204
6,다산콜센터,120다산콜재단,D0000046114132,대외협력과,2022-08-29,경제,○ 하반기 운영일정 구 분 일 정 9월 10월 11월 ...,26695536
7,다산콜센터,120다산콜재단,D0000046546536,서울특별시 푸른도시여가국 공원여가사업과,2022-10-05,행정,"별도 예약이나 신청 절차 없이, 공원에 방문한 누구나 선착순으로 참여가 가능합니다....",27097033
8,다산콜센터,120다산콜재단,D0000046546537,서울특별시 푸른도시여가국 공원여가사업과,2022-10-05,행정,"야외에서 진행되는 행사여서, 비가 많이 오거나 심하게 바람이 부는 등 기상상황에 따...",27097034
9,다산콜센터,120다산콜재단,D0000046828355,요금제도과,2022-11-28,환경,민원인이 세무서 제출용(비용처리 등의 용도) 등으로 수도요금 관련 증빙자료 요청시 ...,27328170


In [35]:
df_detail = df.merge(view_detail, on=['내용번호','생산일'])
df_detail.columns

Index(['번호', '제목', '생산일', '조회수', '내용번호', '원본시스템', '작성자(책임자)', '관리번호', '제공부서',
       '분류', '내용'],
      dtype='object')

In [7]:
file_name = '/home/sjt/projects/web/likelion/AIS8/data/seoul-120/seoul-120-sample.csv'
df_detail = pd.read_csv(file_name)
df_detail

,번호,제목,생산일,조회수,내용번호,원본시스템,작성자(책임자),관리번호,제공부서,분류,내용
0,2234,서울시 [우리동네꼼꼼육아정보] 책자 발간,2019-10-23,2275,18955676,다산콜센터,120다산콜재단,D0000038442669,서울특별시 여성가족정책실 아이돌봄담당관,복지,서울시와 서울시육아종합지원센터는 140여 명의 우리동네 보육반장이 직접 발로 뛰어 ...
1,2235,"[성동구] 반려동물 ""무료상담실"" 운영 안내",2020-08-31,1654,21095911,다산콜센터,120다산콜재단,D0000040706778,서울특별시 자치구,경제,"▣ 반려동물 무료상담실_""무엇이든 물어보시개?"" 운영 안내 구분 내용 ..."
2,2236,위드유 서울 직장 성희롱.성폭력 예방센터,2020-09-16,1924,21212235,다산콜센터,120다산콜재단,D0000040829456,서울특별시 여성가족정책실 여성권익담당관,복지,업무개요 직장 내 성희롱 지원체계 사각지대에 있는 서울 시민이 안전하고 성평등하...
3,2237,마포 뇌병변장애인 비전센터,2021-03-12,1625,22477798,다산콜센터,120다산콜재단,D0000042114742,서울특별시 복지정책실 복지기획관 장애인복지정책과,복지,▣ 마포뇌병변장애인 비전센터 운영 구분 내용 목적 학령기 이후 갈...
4,2238,[서울산업진흥원] 서울메이드란?,2021-06-29,8168,23194045,다산콜센터,120다산콜재단,D0000042894548,서울산업진흥원,경제,서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해 만들어진 라이프스타일 플랫...
5,2239,다자녀가정 실내 바닥매트 지원,2021-08-17,1319,25670204,다산콜센터,120다산콜재단,D0000043274644,서울특별시 서초구 주민생활국 여성보육과,복지,업무개요 다자녀가정 아이들의 안전한 실내 활동과 층간소음 예방을 위해 실내 바닥...
6,2240,하반기 농부의 시장 일정은 어떻게 되나요?,2022-08-29,647,26695536,다산콜센터,120다산콜재단,D0000046114132,대외협력과,경제,○ 하반기 운영일정 구 분 일 정 9월 10월 11월 ...
7,2241,핑크퐁 서울공원 대탐험 행사에 참여하려면 예약이 필요한가요?,2022-10-05,48,27097033,다산콜센터,120다산콜재단,D0000046546536,서울특별시 푸른도시여가국 공원여가사업과,행정,"별도 예약이나 신청 절차 없이, 공원에 방문한 누구나 선착순으로 참여가 가능합니다...."
8,2242,핑크퐁 서울공원 대탐험 행사는 비가 와도 운영하나요?,2022-10-05,129,27097034,다산콜센터,120다산콜재단,D0000046546537,서울특별시 푸른도시여가국 공원여가사업과,행정,"야외에서 진행되는 행사여서, 비가 많이 오거나 심하게 바람이 부는 등 기상상황에 따..."
9,2243,수도요금 전자계산서 발급 요청시,2022-11-28,121,27328170,다산콜센터,120다산콜재단,D0000046828355,요금제도과,환경,민원인이 세무서 제출용(비용처리 등의 용도) 등으로 수도요금 관련 증빙자료 요청시 ...


In [10]:
url = 'https://finance.naver.com/api/sise/etfItemList.nhn?etfType=0&targetColumn=market_sum&sortOrder=desc'

In [15]:
result = pd.read_json(url, encoding='cp949').loc["etfItemList","result"]

In [16]:
pd.DataFrame(result).head()

,itemcode,etfTabCode,itemname,nowVal,risefall,changeVal,changeRate,nav,threeMonthEarnRate,quant,amonut,marketSum
0,069500,1,KODEX 200,31600,5,-245,-0.77,31645.0,9.9126,2369094,74875,55174
1,357870,7,TIGER CD금리투자KIS(합성),51775,2,5,0.01,51779.0,0.9161,63984,3312,44988
2,423160,7,KODEX KOFR금리액티브(합성),101840,3,0,0.00,101850.0,0.8167,29168,2970,31259
3,371460,4,TIGER 차이나전기차SOLACTIVE,12275,2,130,1.07,12305.0,-14.1393,2325067,28438,30354
4,273130,6,KODEX 종합채권(AA-이상)액티브,103775,2,320,0.31,103812.0,5.0304,7224,749,24301


In [18]:
res = requests.get(url)
res.status_code

200

In [23]:
etf_json = res.json()
etfItemList = etf_json['result']['etfItemList']
df = pd.DataFrame(etfItemList)
df

,itemcode,etfTabCode,itemname,nowVal,risefall,changeVal,changeRate,nav,threeMonthEarnRate,quant,amonut,marketSum
0,069500,1,KODEX 200,31605,5,-240,-0.75,31654.0,9.9126,2392354,75610,55182
1,357870,7,TIGER CD금리투자KIS(합성),51775,2,5,0.01,51779.0,0.9161,64608,3345,44988
2,423160,7,KODEX KOFR금리액티브(합성),101845,2,5,0.00,101850.0,0.8167,30106,3066,31260
3,371460,4,TIGER 차이나전기차SOLACTIVE,12270,2,125,1.03,12311.0,-14.1393,2437003,29812,30341
4,273130,6,KODEX 종합채권(AA-이상)액티브,103850,2,395,0.38,103829.0,5.0304,8138,843,24319
...,...,...,...,...,...,...,...,...,...,...,...,...
663,433870,7,ARIRANG TDF2050액티브,9945,2,45,0.45,10110.0,1.1235,41,0,18
664,315480,2,KBSTAR 200커뮤니케이션서비스,11695,2,25,0.21,11737.0,17.2864,112,1,16
665,287330,2,KBSTAR 200생활소비재,6875,5,-35,-0.51,6906.0,17.5170,6243,42,15
666,284990,2,KBSTAR 200에너지화학,10280,5,-95,-0.92,10272.0,7.0691,460,4,12


In [160]:
import io

def get_df_info(df):
    buffer = io.StringIO()
    df.info(buf=buffer)
    s = buffer.getvalue()
    location_column = s.split('\n')[3:-3][0].find('Column')
    location_count = s.split('\n')[3:-3][0].find('Non-Null Count')
    location_dtype = s.split('\n')[3:-3][0].find('Dtype')
    types = s.split('\n')[3:-3][2:]
    list_lows = list()
    for i in types:
        list_lows.append(
            (
                i[location_column:location_count].strip(),
                i[location_count:location_dtype].strip(),
                i[location_dtype:].strip()
            )
        )
        
    df_info = pd.DataFrame(list_lows,columns=['Columns', 'Non-Null Count', 'Dtype'])
    return df_info

def get_df_dtype(df):
    buffer = io.StringIO()
    df.info(buf=buffer)
    s = buffer.getvalue()
    location_column = s.split('\n')[3:-3][0].find('Column')
    location_count = s.split('\n')[3:-3][0].find('Non-Null Count')
    location_dtype = s.split('\n')[3:-3][0].find('Dtype')
    types = s.split('\n')[3:-3][2:]
    list_lows = list()
    for i in types:
        list_lows.append(
            (
                {i[location_column:location_count].strip():i[location_dtype:].strip()}
            )
        )
    return list_lows

def save_df_with_dtype(path,df):
    with open(path,'w') as f:
        f.write(f'# {get_df_dtype(df)} \n')
        f.write(df.to_csv(index=False))

In [78]:
display(
    get_df_info(df),
    df.describe().T,
    df.describe(include=object),
    df.nlargest(10, 'amonut'),
    df.nsmallest(10, 'amonut')
    )


,Columns,Non-Null Count,Dtype
0,itemcode,668 non-null,object
1,etfTabCode,668 non-null,int64
2,itemname,668 non-null,object
3,nowVal,668 non-null,int64
4,risefall,668 non-null,object
5,changeVal,668 non-null,int64
6,changeRate,668 non-null,float64
7,nav,668 non-null,float64
8,threeMonthEarnRate,622 non-null,float64
9,quant,668 non-null,int64


,count,mean,std,min,25%,50%,75%,max
etfTabCode,668.0,3.564371,1.905658e+00,1.0000,2.00000,4.00000,5.0000,7.000000e+00
nowVal,668.0,20035.748503,2.643135e+04,2440.0000,8481.25000,10332.50000,14788.7500,2.015950e+05
changeVal,668.0,17.627246,1.847453e+02,-625.0000,-65.00000,0.00000,66.2500,1.745000e+03
changeRate,668.0,0.063323,9.529259e-01,-2.5800,-0.66000,0.00000,0.6600,5.710000e+00
nav,668.0,20039.288922,2.642603e+04,517.0000,8488.25000,10339.00000,14831.0000,2.015900e+05
threeMonthEarnRate,622.0,5.482033,1.027639e+01,-25.4072,0.04705,6.01105,10.2054,8.074470e+01
quant,668.0,358540.455090,4.993991e+06,0.0000,189.75000,2233.00000,20478.5000,1.237736e+08
amonut,668.0,2385.899701,1.961939e+04,0.0000,2.00000,30.50000,250.5000,3.738430e+05
marketSum,668.0,1254.306886,4.222347e+03,11.0000,71.75000,147.00000,604.5000,5.518200e+04


,itemcode,itemname,risefall
count,668,668,668
unique,668,668,3
top,069500,KODEX 200,2
freq,1,1,323


In [98]:
from datetime import datetime

today_date = datetime.today().strftime('%Y-%m-%d')
file_name = f'etf-{today_date}.csv'
df.to_csv(f'../../data/naver-finance/{file_name}',index=False)

'2023-01-18'

In [104]:
df = pd.read_csv(f'../../data/naver-finance/{file_name}',dtype={'itemcode':'O'})
df

In [122]:
# "서울시 다산콜센터 자주묻는질문 목록" 데이터의 메타정보를 담은 페이지를 url 변수에 담아줍니다.
# pd.read_html 을 통해 해당 URL의 table 정보를 읽어옵니다.
# DataFrame의 0행을 인덱스로 만듭니다.
# 행을 열으로, 열을 행으로 작성하여 바꾸어줍니다. pandas.DataFrame.transpose와 동일합니다.
api_document_url = "https://data.seoul.go.kr/dataList/OA-1127/S/1/datasetView.do"
df_api_doc = pd.read_html(api_document_url)[2].set_index(0).T
print(api_document_url)
df_api_doc

# df_api_doc 1행의 라이선스를 출력합니다.
df_api_doc["라이선스"]

https://data.seoul.go.kr/dataList/OA-1127/S/1/datasetView.do


1    저작권자표시(BY) : 이용이나 변경 및 2차적 저작물의 작성을 포함한 자유이용을 ...
Name: 라이선스, dtype: object

In [126]:
with open('/home/sjt/projects/web/likelion/data/apikeys/seoul-120.txt','r') as f:
    auth_key = f.read()

faq_url = f"http://openAPI.seoul.go.kr:8088/{auth_key}/xml/SearchFAQService/1/999/F"

result = requests.get(faq_url)

result.text

'<?xml version="1.0" encoding="UTF-8"?>\n<SearchFAQService>\n<list_total_count>21</list_total_count>\n<RESULT>\n<CODE>INFO-000</CODE>\n<MESSAGE>정상 처리되었습니다</MESSAGE>\n</RESULT>\n<row>\n<FAQ_TP>F</FAQ_TP>\n<FAQ_SEQNO>289427</FAQ_SEQNO>\n<FAQ_TP_NAME>FAQ</FAQ_TP_NAME>\n<LCODE>22214327</LCODE>\n<LCODE_NAME>일반공공행정</LCODE_NAME>\n<QUEST><![CDATA[[시ㆍ구정외 타기관 관련 상담] 고용노동부 [일자리 안정자금]]]></QUEST>\n<UPDATE_YMDHMS>20190522102043</UPDATE_YMDHMS>\n</row>\n<row>\n<FAQ_TP>F</FAQ_TP>\n<FAQ_SEQNO>289434</FAQ_SEQNO>\n<FAQ_TP_NAME>FAQ</FAQ_TP_NAME>\n<LCODE>22214061</LCODE>\n<LCODE_NAME>문화와여가</LCODE_NAME>\n<QUEST><![CDATA[서대문문화체육회관 FAQ]]></QUEST>\n<UPDATE_YMDHMS>20190816162025</UPDATE_YMDHMS>\n</row>\n<row>\n<FAQ_TP>F</FAQ_TP>\n<FAQ_SEQNO>289435</FAQ_SEQNO>\n<FAQ_TP_NAME>FAQ</FAQ_TP_NAME>\n<LCODE>22214061</LCODE>\n<LCODE_NAME>문화와여가</LCODE_NAME>\n<QUEST><![CDATA[서대문구립인조잔디구장 FAQ]]></QUEST>\n<UPDATE_YMDHMS>20190816162025</UPDATE_YMDHMS>\n</row>\n<row>\n<FAQ_TP>F</FAQ_TP>\n<FAQ_SEQNO>289436</FAQ_SEQNO>\n<FAQ_TP_N

In [144]:
df_list_all = pd.read_xml(result.text)
df_list_all

,list_total_count,CODE,MESSAGE,FAQ_TP,FAQ_SEQNO,FAQ_TP_NAME,LCODE,LCODE_NAME,QUEST,UPDATE_YMDHMS
0,21.0,None,None,None,NaN,None,NaN,None,None,NaN
1,NaN,INFO-000,정상 처리되었습니다,None,NaN,None,NaN,None,None,NaN
2,NaN,None,None,F,289427.0,FAQ,22214327.0,일반공공행정,[시ㆍ구정외 타기관 관련 상담] 고용노동부 [일자리 안정자금],2.019052e+13
3,NaN,None,None,F,289434.0,FAQ,22214061.0,문화와여가,서대문문화체육회관 FAQ,2.019082e+13
4,NaN,None,None,F,289435.0,FAQ,22214061.0,문화와여가,서대문구립인조잔디구장 FAQ,2.019082e+13
5,NaN,None,None,F,289436.0,FAQ,22214061.0,문화와여가,궁동체육관 FAQ,2.019082e+13
6,NaN,None,None,F,289437.0,FAQ,22214061.0,문화와여가,홍제배드민턴장 FAQ,2.019082e+13
7,NaN,None,None,F,289472.0,FAQ,22214153.0,재난과안전,양천생활안전체험교육관,2.018021e+13
8,NaN,None,None,F,289473.0,FAQ,22214061.0,문화와여가,금천 나도스타 노래부르기 대회 (어린이날 행사),2.019050e+13
9,NaN,None,None,F,289492.0,FAQ,22214061.0,문화와여가,브라보 서초문화버스 (셔틀버스) 운행,2.018113e+13


In [186]:
df_list = df_list_all.iloc[2:,3:].reset_index(drop=True)
df_list

,FAQ_TP,FAQ_SEQNO,FAQ_TP_NAME,LCODE,LCODE_NAME,QUEST,UPDATE_YMDHMS
0,F,289427.0,FAQ,22214327.0,일반공공행정,[시ㆍ구정외 타기관 관련 상담] 고용노동부 [일자리 안정자금],2.019052e+13
1,F,289434.0,FAQ,22214061.0,문화와여가,서대문문화체육회관 FAQ,2.019082e+13
2,F,289435.0,FAQ,22214061.0,문화와여가,서대문구립인조잔디구장 FAQ,2.019082e+13
3,F,289436.0,FAQ,22214061.0,문화와여가,궁동체육관 FAQ,2.019082e+13
4,F,289437.0,FAQ,22214061.0,문화와여가,홍제배드민턴장 FAQ,2.019082e+13
5,F,289472.0,FAQ,22214153.0,재난과안전,양천생활안전체험교육관,2.018021e+13
6,F,289473.0,FAQ,22214061.0,문화와여가,금천 나도스타 노래부르기 대회 (어린이날 행사),2.019050e+13
7,F,289492.0,FAQ,22214061.0,문화와여가,브라보 서초문화버스 (셔틀버스) 운행,2.018113e+13
8,F,289612.0,FAQ,22214339.0,지역개발,마곡산업단지 내 문화시설 건립,2.018091e+13
9,F,289648.0,FAQ,22214328.0,농림해양수산,서울반려동물교육센터,2.019041e+13


In [187]:
df_list["UPDATE_YMD"] = pd.to_datetime(df_list["UPDATE_YMDHMS"].astype(str).str[:14])
df_list

,FAQ_TP,FAQ_SEQNO,FAQ_TP_NAME,LCODE,LCODE_NAME,QUEST,UPDATE_YMDHMS,UPDATE_YMD
0,F,289427.0,FAQ,22214327.0,일반공공행정,[시ㆍ구정외 타기관 관련 상담] 고용노동부 [일자리 안정자금],2.019052e+13,2019-05-22 10:20:43
1,F,289434.0,FAQ,22214061.0,문화와여가,서대문문화체육회관 FAQ,2.019082e+13,2019-08-16 16:20:25
2,F,289435.0,FAQ,22214061.0,문화와여가,서대문구립인조잔디구장 FAQ,2.019082e+13,2019-08-16 16:20:25
3,F,289436.0,FAQ,22214061.0,문화와여가,궁동체육관 FAQ,2.019082e+13,2019-08-16 16:20:14
4,F,289437.0,FAQ,22214061.0,문화와여가,홍제배드민턴장 FAQ,2.019082e+13,2019-08-16 16:20:34
5,F,289472.0,FAQ,22214153.0,재난과안전,양천생활안전체험교육관,2.018021e+13,2018-02-13 14:50:25
6,F,289473.0,FAQ,22214061.0,문화와여가,금천 나도스타 노래부르기 대회 (어린이날 행사),2.019050e+13,2019-05-02 16:00:33
7,F,289492.0,FAQ,22214061.0,문화와여가,브라보 서초문화버스 (셔틀버스) 운행,2.018113e+13,2018-11-28 12:09:32
8,F,289612.0,FAQ,22214339.0,지역개발,마곡산업단지 내 문화시설 건립,2.018091e+13,2018-09-10 13:15:34
9,F,289648.0,FAQ,22214328.0,농림해양수산,서울반려동물교육센터,2.019041e+13,2019-04-08 17:51:05


In [180]:
faq_no = 289435
detail_url = f"http://openapi.seoul.go.kr:8088/sample/xml/SearchDetailsFAQService/1/5/F/{faq_no}"

cont = pd.read_xml(detail_url)
cont

,list_total_count,CODE,MESSAGE,FAQ_TP,FAQ_SEQNO,QUEST,ANSWER,UPDATE_YMDHMS
0,1.0,None,None,None,NaN,None,None,NaN
1,NaN,INFO-000,정상 처리되었습니다,None,NaN,None,None,NaN
2,NaN,None,None,F,289435.0,서대문구립인조잔디구장 FAQ,？？\n\n\n\n\n\n주요 질의사항\n\n\n\n답 변 사 항\n\n\n\n\n...,2.018012e+13


In [188]:
FAQ_TP = df_list.iloc[0]['FAQ_TP']
FAQ_SEQNO = df_list.iloc[0]['FAQ_SEQNO']

detail_url = f'http://openAPI.seoul.go.kr:8088/{auth_key}/xml/SearchDetailsFAQService/1/1/{FAQ_TP}/{FAQ_SEQNO}/'

result = pd.read_xml(detail_url).iloc[-1]['ANSWER']
result

'■ 최저임금 해결사 일자리 안정자금\n\n ○ 정부(고용노동부)에서 &quot;일자리 안정자금&quot;을 최저임금 인상으로 부담커진 소상공인. 영세사업주에 지원\n\n  ※ 처리구분 및 상담유형 : [유관기관] - [노동부(고용/근로)]\n\n\n\n\n\n\u3000\n\n\n\n구분\n\n\n\n설명\n\n\n\n\n\n\u3000\n\n\n\n신청시기\n\n\n\n 연 1회 (2018년 1월 1일~12월 31일)\n\n ※ 사업시행일 이후 연중 1회 신청으로 최초 지원대상월부터 18년말 임금지급월까지 지급 (신청월 이후 매월 자동 지급)\n\n\n\n\n\n\u3000\n\n\n\n대상자\n\n\n\n지원대상\n\n\n\n ○ 노동자를 30인 미만으로 고용하는 모든 사업(주)에 대해 지원\n\n  ※ [예외] 공동주택 경비.청소원에 대해서는 30인 이상 사업주도 지원\n\n\n\n\n\n\u3000\n\n\n\n지원제외\n\n\n\n 소득 사업주(과세소득 5억 초과), 임금체불 명단 공개 사업주, 국가 등 공공부문, 국가 등으로부터 인건비 재정지원 받고 있는 사업주 등\n\n\n\n\n\n\u3000\n\n\n\n지원요건\n\n\n\n ○ 신청일 이전 1개월 이상 고용이 유지된 월 보수액 210만원 미만 노동자\n\n ○ 최저임금 준수 및 고용보험 가입 원칙\n\n ○ 기존 노동자 임금(보수) 수준 저하 금지 및 고용유지 의무\n\n\n\n\n\n\u3000\n\n\n\n접수처\n\n\n\n온라인\n\n\n\n  근로복지공단 고용.산재토탈서비스 (http://total.kcomwel.or.kr)\n\n\n\n\n\n\u3000\n\n\n\n팩스.방문\n\n\n\n  근로복지공단 관할지사\n\n\n\n\n\n\u3000\n\n\n\n지원금액\n\n\n\n 노동자 1인당 월 13만원 (단, 주 40시간 미만 근로자는 근로시간 비례 지급)\n\n\n\n  ※ 5인미만 사업장 월15만원 지원\n\n\n\n\n\n\u3000\n\n\n\n지급방식\n\n\n\n

In [193]:
def get_content(FAQ_SEQNO):
    detail_url = f'http://openAPI.seoul.go.kr:8088/{auth_key}/xml/SearchDetailsFAQService/1/1/F/{FAQ_SEQNO}/'
    result = pd.read_xml(detail_url).iloc[2:]['ANSWER']
    return result

In [195]:
get_content(df_list['FAQ_SEQNO'][0])

2    ■ 최저임금 해결사 일자리 안정자금\n\n ○ 정부(고용노동부)에서 &quot;일자...
Name: ANSWER, dtype: object

In [194]:
df_list['ANSWER'] = df_list['FAQ_SEQNO'].progress_apply(get_content)
df_list

100%|██████████| 21/21 [00:01<00:00, 11.22it/s]


,FAQ_TP,FAQ_SEQNO,FAQ_TP_NAME,LCODE,LCODE_NAME,QUEST,UPDATE_YMDHMS,UPDATE_YMD,ANSWER
0,F,289427.0,FAQ,22214327.0,일반공공행정,[시ㆍ구정외 타기관 관련 상담] 고용노동부 [일자리 안정자금],2.019052e+13,2019-05-22 10:20:43,■ 최저임금 해결사 일자리 안정자금\n\n ○ 정부(고용노동부)에서 &quot;일자...
1,F,289434.0,FAQ,22214061.0,문화와여가,서대문문화체육회관 FAQ,2.019082e+13,2019-08-16 16:20:25,？？\n\n\n\n\n\n주요 질의사항\n\n\n\n답 변 사 항\n\n\n\n\n...
2,F,289435.0,FAQ,22214061.0,문화와여가,서대문구립인조잔디구장 FAQ,2.019082e+13,2019-08-16 16:20:25,？？\n\n\n\n\n\n주요 질의사항\n\n\n\n답 변 사 항\n\n\n\n\n...
3,F,289436.0,FAQ,22214061.0,문화와여가,궁동체육관 FAQ,2.019082e+13,2019-08-16 16:20:14,？？\n\n\n\n\n\n주요 질의사항\n\n\n\n답 변 사 항\n\n\n\n\n...
4,F,289437.0,FAQ,22214061.0,문화와여가,홍제배드민턴장 FAQ,2.019082e+13,2019-08-16 16:20:34,？？\n\n\n\n\n\n주요 질의사항\n\n\n\n답 변 사 항\n\n\n\n\n...
5,F,289472.0,FAQ,22214153.0,재난과안전,양천생활안전체험교육관,2.018021e+13,2018-02-13 14:50:25,구분\n\n\n\n내용\n\n\n\n\n\n업무개요\n\n\n\n 최첨단 교육 실습...
6,F,289473.0,FAQ,22214061.0,문화와여가,금천 나도스타 노래부르기 대회 (어린이날 행사),2.019050e+13,2019-05-02 16:00:33,▣ [금천구] 제9회 금천 &apos;나도스타&apos; 노래부르기 대회 참가자 모...
7,F,289492.0,FAQ,22214061.0,문화와여가,브라보 서초문화버스 (셔틀버스) 운행,2.018113e+13,2018-11-28 12:09:32,▣ 업무개요\n\n\n\n교통취약지역과 관내 문화시설을 연계하여 이용하는 서초구민의...
8,F,289612.0,FAQ,22214339.0,지역개발,마곡산업단지 내 문화시설 건립,2.018091e+13,2018-09-10 13:15:34,업무개요\n\n\n\n 마곡산업단지 내 문화시설 건립\n\n\n\n\n\n 업무설명...
9,F,289648.0,FAQ,22214328.0,농림해양수산,서울반려동물교육센터,2.019041e+13,2019-04-08 17:51:05,교육\n\n\n\n주제\n\n\n\n일시\n\n\n\n횟수 및 모집인원\n\n\n\...
